# Used Car Price Prediction

[Kaggle Resource](https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset)

After countless total hours... probably even total days of learning, studying, preparation, examination of best practices, and relearning... it's finally time to get down to business.

I've also recently decided that you can only do so much preparation for a complicated field like ML, so I'm going to likely have to fill in some gaps along the way here. Thankfully, Kaggle has a LOT of publicly posted code to assist me here. I will abstain from referencing it as long as I can though, and will make sure to give credit where it's due.

Anyway, I believe this data marks a perfect balance between a challenging and approachable project. Let's begin.

# Load and Examine Data

In [315]:
import pandas as pd
import re
import numpy as np
import sklearn as skl
import matplotlib.pyplot as plt

In [316]:
cars = pd.read_csv("data/used_cars.csv")

Let's just get a quick overview of some basic precleaning stuff.

In [317]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


In [318]:
print(f"Missing Values:\n{cars.isna().sum()}\n\nTotal Duplicates:\n{cars.duplicated().sum()}")

Missing Values:
brand             0
model             0
model_year        0
milage            0
fuel_type       170
engine            0
transmission      0
ext_col           0
int_col           0
accident        113
clean_title     596
price             0
dtype: int64

Total Duplicates:
0


Now that that's out of the way, let's actually take a peak at the data now.

In [319]:
cars.head(10)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"
5,Acura,ILX 2.4L,2016,"136,397 mi.",Gasoline,2.4 Liter,F,Silver,Ebony.,None reported,NaN,"$14,798"
6,Audi,S3 2.0T Premium Plus,2017,"84,000 mi.",Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Blue,Black,None reported,Yes,"$31,000"
7,BMW,740 iL,2001,"242,000 mi.",Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,A/T,Green,Green,None reported,Yes,"$7,300"
8,Lexus,RC 350 F Sport,2021,"23,436 mi.",Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,6-Speed A/T,Black,Black,None reported,Yes,"$41,927"
9,Tesla,Model X Long Range Plus,2020,"34,000 mi.",NaN,534.0HP Electric Motor Electric Fuel System,A/T,Black,Black,None reported,Yes,"$69,950"


# Clean the Data

Now that the preliminary stuff is out of the way, let's start cleaning the data up.

It'd be best to start simple here and worry about the basic numeric conversion first. We'll start by removing unnecessary string values from columns that will be designated as strictly numeric. I also think it'd be interesting to see if there are any differences in the model outputs when using `float32` instead of `float64`. We'll get to that much later though.

For now, let's start with simple data cleansing. Then we can chip away at the more tricky regex stuff and increase the complexity as we get a feel for the data.

In [320]:
# Remove non integer string values
cars[["milage", "price"]] = cars[["milage", "price"]].replace(r'[^\d]', '', regex=True)    


# `engine` column split
cars["horsepower"] = cars["engine"].str.extract(r'([\d.]+)HP', expand=False)
cars["liters"] = cars["engine"].str.extract(r'([\d.]+)L', expand=False)


# `fuel_type` column split
cars["gas_diesel_other"] = (
    cars["fuel_type"].str.contains("gasoline", case=False, na=False) |
    cars["fuel_type"].str.contains("diesel", case=False, na=False) |
    cars["fuel_type"].str.contains("fuel", case=False, na=False)
)
cars["hybrid"] = cars["fuel_type"].str.contains("hybrid", case=False, na=False)
cars["electric"] = cars["fuel_type"].isna()

print(f"Conflicting fuel type values? {(cars[['gas_diesel_other', 'hybrid', 'electric']].sum(axis=1) > 1).any()}")


# `transmission` column split
cars["automatic_transmission"] = (
    cars["transmission"].str.contains("A/T", case=False) |
    cars["transmission"].str.contains("AT", case=False) |
    cars["transmission"].str.contains("automatic", case=False) |
    cars["transmission"].str.contains("cvt", case=False)
)
cars['dual_shift_transmission'] = cars['transmission'].str.contains("dual", case=False)
cars["manual_transmission"] = (
    cars['transmission'].str.contains("manual", case=False) |
    cars['transmission'].str.contains("m/t", case=False) |
    cars['transmission'].str.contains("mt", case=False)
)
cars.loc[cars['dual_shift_transmission'], ['automatic_transmission', 'manual_transmission']] = False
cars.loc[cars['manual_transmission'], ['automatic_transmission', 'dual_shift_transmission']] = False

print(f"Conflicting transmission values? {(cars[['automatic_transmission', 'manual_transmission', "dual_shift_transmission"]].sum(axis=1) > 1).any()}")


# `accident` column split

cars[["model", "transmission", "automatic_transmission", "manual_transmission", "dual_shift_transmission"]]

Conflicting fuel type values? False
Conflicting transmission values? False


,model,transmission,automatic_transmission,manual_transmission,dual_shift_transmission
0,Utility Police Interceptor Base,6-Speed A/T,True,False,False
1,Palisade SEL,8-Speed Automatic,True,False,False
2,RX 350 RX 350,Automatic,True,False,False
3,Q50 Hybrid Sport,7-Speed A/T,True,False,False
4,Q3 45 S line Premium Plus,8-Speed Automatic,True,False,False
...,...,...,...,...,...
4004,Continental GT Speed,8-Speed Automatic with Auto-Shift,True,False,False
4005,S4 3.0T Premium Plus,Transmission w/Dual Shift Mode,False,False,True
4006,Taycan,Automatic,True,False,False
4007,F-150 Raptor,A/T,True,False,False


# Fill in Missing Values

Now that the more simple stuff is out of the way, let's start filling in those missing values. The way I see it we have two options: We can either take the mean values for the relevant column of eac h brand, or run some separate regressions to potentially get some more accurate values to fill in. Let's do both and see the results.

In [321]:


# Alternative: using .all() method
cars[(cars[['automatic_transmission', 'dual_shift_transmission', 'manual_transmission']] == False).all(axis=1)]

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horsepower,liters,gas_diesel_other,hybrid,electric,automatic_transmission,dual_shift_transmission,manual_transmission
5,Acura,ILX 2.4L,2016,136397,Gasoline,2.4 Liter,F,Silver,Ebony.,None reported,NaN,14798,NaN,NaN,True,False,False,False,False,False
216,Acura,Integra w/A-Spec Tech Package,2023,3574,Gasoline,1.5 Liter Turbo,Variable,Apex Blue,Orchid,None reported,NaN,34998,NaN,NaN,True,False,False,False,False,False
269,Acura,TLX w/A-Spec Package,2022,14896,Gasoline,2.0 Liter DOHC Turbo,2,Lunar Silver Metallic,Ebony,None reported,NaN,39998,NaN,NaN,True,False,False,False,False,False
476,Acura,MDX w/Technology Package,2023,3415,Gasoline,3.5 Liter SOHC,F,White,Parchment.,None reported,NaN,54998,NaN,NaN,True,False,False,False,False,False
516,Acura,MDX w/Technology Package,2022,30177,Gasoline,3.5L 24V SOHC I-VTEC V6,2,Majestic Black Pearl,Espresso,None reported,NaN,46598,NaN,3.5,True,False,False,False,False,False
536,Porsche,911 Carrera S,2017,16701,Gasoline,3.0 Liter,–,Red,–,None reported,NaN,107998,NaN,NaN,True,False,False,False,False,False
546,Land,Rover Range Rover Sport HSE,2013,81648,Gasoline,"5L V-8 gasoline direct injection, DOHC, variab...","Auto, 6-Spd w/CmdShft",Fuji White,–,None reported,NaN,15900,NaN,5,True,False,False,False,False,False
554,GMC,Sierra 2500 SLE,2015,116210,E85 Flex Fuel,360.0HP 6.0L 8 Cylinder Engine Flex Fuel Capab...,Transmission Overdrive Switch,Black,Beige,None reported,Yes,31900,360.0,6.0,True,False,False,False,False,False
665,Chevrolet,Suburban LT,2017,195361,Gasoline,355.0HP 5.3L 8 Cylinder Engine Gasoline Fuel,Transmission Overdrive Switch,Black,Black,At least 1 accident or damage reported,Yes,26000,355.0,5.3,True,False,False,False,False,False
818,Mercury,Marauder Base,2004,56600,Gasoline,302.0HP 4.6L 8 Cylinder Engine Gasoline Fuel,Transmission Overdrive Switch,Silver,Black,None reported,Yes,23000,302.0,4.6,True,False,False,False,False,False
